In [2]:
import pandas as pd
import numpy as np
import pickle

## Usage Data Frame
usage_container.csv table contains all resources consumed by containers in different machines.

In [3]:
usage_df = pd.read_csv('dataset/usageshort2.csv', names=[
    "container_id",
    "machine_id",
    "time_stamp",
    "cpu_util_percent",
    "mem_util_percent",
    "cpi",
    "mem_gps",
    "mpki",
    "net_in",
    "net_out",
    "disk_io_percent"],
    delimiter=','
)

usage_df.head()

FileNotFoundError: [Errno 2] No such file or directory: 'dataset/usageshort2.csv'

## Meta Data Frame
meta_containers.csv contains information about the containers in the creation time.

In [56]:
meta_df = pd.read_csv('dataset/container_meta.csv',
                   names = [
                       "container_id", 
                       "machine_id", 
                       "time_stamp", 
                       "app_du", 
                       "status",
                       "cpu_request",
                       "cpu_limit",
                       "mem_size"
                   ],
                   delimiter=',')

meta_df.head()

# HACK: 
containers = ["c_10000",
"c_10001",
"c_10002",
"c_10003",
"c_10004",
"c_10005",
"c_10006",
"c_10007",
'c_10008',
"c_10009",
"c_1001",
"c_10010",
"c_10011",
'c_10012',
"c_10013",
"c_10014",
"c_10015",
"c_10016",
"c_10017",
'c_10019',
"c_1002",
"c_10020",
"c_10023",
"c_10024",
"c_10025",
"c_10026",
"c_10027",
"c_10028",
"c_10029",
"c_1003",
"c_10030",
"c_10032",
"c_10033",
"c_10034",
"c_10035",
"c_10036",
"c_10037",
"c_10038",
"c_10039",
"c_1004",
"c_10040",
"c_10041",
"c_10042",
"c_10043",
"c_10044",
"c_10045",
"c_10046",
"c_10047",
"c_10048",
"c_10049",
"c_1005"]

meta_df = meta_df[meta_df.container_id.isin(containers)]
meta_df = meta_df.drop_duplicates(["container_id"])
meta_df.count()
meta_df.tail(20)

,container_id,machine_id,time_stamp,app_du,status,cpu_request,cpu_limit,mem_size
51852,c_10029,m_3068,0,app_9237,started,400,400,1.56
51858,c_10030,m_2259,0,app_302,started,800,800,3.13
51867,c_10032,m_34,0,app_8799,started,800,800,3.13
51874,c_10033,m_2822,0,app_859,started,400,400,1.56
51879,c_10034,m_447,0,app_2603,started,400,400,1.56
51886,c_10035,m_1443,0,app_4186,started,400,400,3.13
51891,c_10036,m_3129,0,app_131,started,800,800,3.13
51896,c_10037,m_1132,0,app_263,started,800,800,3.13
51902,c_10038,m_4005,0,app_1962,started,400,400,1.56
51909,c_10039,m_221,0,app_8339,started,400,400,1.56


In [57]:
# set index for container id
usage_df.set_index("container_id")
usage_df.head()

,container_id,machine_id,time_stamp,cpu_util_percent,mem_util_percent,cpi,mem_gps,mpki,net_in,net_out,disk_io_percent
0,c_10000,m_2113,741020,4,50,1.72,0.0,0.0,0.02,0.03,5.0
1,c_10000,m_2113,743510,6,50,1.72,0.0,0.0,0.02,0.03,9.0
2,c_10000,m_2113,743540,5,50,1.40,0.0,0.0,0.02,0.03,11.0
3,c_10000,m_2113,745150,6,50,1.63,0.0,0.0,0.02,0.03,14.0
4,c_10000,m_2113,745200,5,50,1.94,0.0,1.0,0.02,0.03,16.0


In [76]:
final = {}

for _, c in meta_df.iterrows():
    
    result = {}
    
    filter_df = usage_df[usage_df.container_id.isin([c['container_id']])]
    
#     result["workload"] = [filter_df[["mem_util_percent"]].to_numpy().flatten(),
#                           filter_df[["cpu_util_percent"]].to_numpy().flatten()]
    
#     result["workload"] = [filter_df[["mem_util_percent"]].to_numpy().flatten(),
#                           filter_df[["cpu_util_percent"]].to_numpy().flatten()]
    result["workload"] = [filter_df[["mem_util_percent", "cpu_util_percent"]].to_numpy()]

#     print([filter_df[["cpu_util_percent"]].to_numpy().flatten(),
#            filter_df[["mem_util_percent"]].to_numpy().flatten()])
  
#     result["timestamps"] = filter_df[["time_stamp"]].to_numpy().flatten()
    result["timestamps"] = filter_df[["time_stamp"]].to_numpy().flatten() - np.amin(filter_df[["time_stamp"]].to_numpy().flatten())

    result["request"] = [c[["mem_size"]].values[0],
                        c[["cpu_request"]].values[0] // 100]
    
    result["limit"] = [c[["mem_size"]].values[0],
                       c[["cpu_limit"]].values[0] // 100]
    
    final[c["container_id"]] = result

# print(final)

# Save results in a pickle file

In [78]:
with open('alibaba.pickle', 'wb') as handle:
    pickle.dump(final, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [77]:
result_df = pd.DataFrame.from_dict(final)
result_df.head()


,c_1001,c_1002,c_1003,c_1004,c_1005,c_10000,c_10001,c_10002,c_10003,c_10004,...,c_10040,c_10041,c_10042,c_10043,c_10044,c_10045,c_10046,c_10047,c_10048,c_10049
workload,"[[[100, 14], [98, 14], [99, 13], [100, 12], [9...","[[[100, 12], [100, 12], [98, 6], [99, 7], [98,...","[[[100, 13], [100, 10], [100, 14], [100, 13], ...","[[[74, 4], [74, 2], [74, 3], [75, 2], [75, 3],...","[[[99, 9], [99, 10], [99, 10], [99, 11], [100,...","[[[50, 4], [50, 6], [50, 5], [50, 6], [50, 5],...","[[[99, 6], [99, 5], [99, 6], [100, 5], [100, 5...","[[[86, 4], [79, 3], [80, 3], [80, 3], [80, 3],...","[[[99, 17], [100, 18], [99, 13], [100, 10], [1...","[[[79, 7], [80, 9], [81, 6], [81, 5], [82, 8],...",...,"[[[95, 4], [95, 4], [95, 7], [96, 5], [93, 4],...","[[[14, 1], [14, 1], [14, 2], [14, 1], [14, 1],...","[[[87, 5], [86, 6], [86, 6], [85, 4], [85, 13]...","[[[49, 4], [50, 2], [50, 4], [50, 2], [50, 5],...","[[[100, 33], [100, 37], [99, 34], [99, 24], [1...","[[[100, 11], [99, 14], [100, 10], [100, 10], [...","[[[56, 0], [56, 0], [56, 0], [56, 0], [56, 0],...","[[[54, 2], [54, 4], [54, 18], [53, 2], [53, 2]...","[[[100, 10], [100, 8], [99, 9], [100, 8], [100...","[[[99, 14], [100, 11], [100, 4], [99, 3], [100..."
timestamps,"[0, 1090, 1180, 1610, 2400, 5670, 5730, 6350, ...","[0, 180, 4030, 4710, 5010, 5640, 5870, 6600, 6...","[0, 520, 820, 1030, 3050, 4640, 4760, 5950, 68...","[0, 90, 580, 1170, 2230, 2450, 4530, 4580, 484...","[0, 860, 1020, 1080, 3020, 3840, 4680, 5440, 5...","[0, 2490, 2520, 4130, 4180, 5730, 6240, 8130, ...","[0, 520, 1120, 1680, 2880, 2990, 3590, 3770, 4...","[0, 4190, 4610, 4710, 4830, 5230, 5600, 7260, ...","[0, 170, 600, 2870, 3000, 3300, 5630, 6900, 72...","[0, 490, 1240, 1890, 1990, 2540, 3030, 3330, 3...",...,"[0, 2460, 2490, 3220, 4240, 4270, 4330, 5120, ...","[0, 1420, 2660, 4210, 4690, 5250, 5450, 6540, ...","[0, 690, 810, 2390, 2500, 4350, 4610, 7230, 75...","[0, 870, 900, 1710, 1800, 2000, 2140, 2420, 25...","[0, 50, 1530, 3860, 4070, 4080, 4460, 4800, 62...","[0, 50, 180, 310, 430, 1240, 2090, 2340, 2710,...","[0, 360, 15930, 16100, 17000, 33160, 39030, 39...","[0, 310, 1270, 2850, 4310, 4560, 5240, 5540, 5...","[0, 700, 2100, 3850, 3960, 4800, 5650, 6170, 6...","[0, 790, 7910, 9290, 9690, 9730, 11290, 11910,..."
request,"[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]",...,"[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.95, 4]","[1.56, 4]","[1.56, 4]","[1.95, 4]","[1.56, 4]"
limit,"[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]",...,"[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.56, 4]","[1.95, 4]","[1.56, 4]","[1.56, 4]","[1.95, 4]","[1.56, 4]"


In [102]:
m = usage_df.drop_duplicates(["container_id"])

In [27]:
# for _, i in m[["container_id"]].iterrows():
#      print(i.values[0])

In [26]:
# a = pd.DataFrame.from_dict(final)
# a.head()